In [1]:
import os
import pickle
os.chdir('/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2')

from datamodule.preprocessing import *
from model.metric import *
os.chdir('/kaggle/working')

2024-06-25 18:12:21.334217: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 18:12:21.334361: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 18:12:21.508887: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import polars as pl

train_df = pl.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
train_df.shape

(17307, 3)

# DeBerta Features

In [3]:
def create_llm_features(df, model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSequenceClassification.from_pretrained(model_id)

    df = df.to_pandas()
    dataset = Dataset.from_pandas(df)
    dataset = dataset.map(
        tokenizer,
        fn_kwargs={'truncation': True, 'max_length': 1024},
        input_columns='full_text'
    )

    trainer = Trainer(
        model=model,
        tokenizer=tokenizer
    )

    predictions = trainer.predict(dataset).predictions
    llm_df = pd.DataFrame({
        f'{model_id}_label_{column_index}': predictions[:, column_index]
        for column_index in range(predictions.shape[1]) 
    })

    return llm_df

In [4]:
model_directory = '/kaggle/input/deberta-training-example'
model_id = 'deberta-v3-small_fold_{fold_index}'
llm_df = []

for fold_index in range(5):
    model_path = os.path.join(model_directory, model_id.format(fold_index=fold_index))
    llm_df.append(create_llm_features(train_df, model_path))

essay_ids = train_df['essay_id'].to_pandas()
llm_df = pd.concat(llm_df + [essay_ids], axis=1)
llm_df.head()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

,/kaggle/input/deberta-training-example/deberta-v3-small_fold_0_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_1_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_2_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_3_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_4_label_0,essay_id
0,1.337985,1.402706,0.910786,1.157639,0.956462,000d118
1,1.775424,2.029668,1.919433,2.041271,1.949009,000fe60
2,3.398520,3.368047,3.679338,3.384858,3.397960,001ab80
3,2.694017,2.828226,2.725597,2.795437,2.752751,001bdc0
4,1.841379,1.429684,1.674938,1.622078,1.779500,002ba53


# Create Word Features

In [5]:
def create_word_features(df):
    df = df.with_columns(
        pl.col('full_text')
        .map_elements(sanitize_text)
        .str.split(by=' ').
        alias('word')
    )

    df = df.explode('word')

    df = df.with_columns(
        pl.col('word')
        .map_elements(lambda x: len(x))
        .alias('word_length')
    )

    df = df.filter(pl.col('word_length') != 0)

    # Aggregate features
    feature_aggregates = []
    feature_aggregates += [
        pl.col('word')
        .filter(pl.col('word_length') >= i + 1)
        .count()
        .alias(f'word_length_greater_equal_{i + 1}')
        for i in range(10)
    ]

    feature_aggregates += [
        pl.col('word_length').max().alias(f'word_length_max'),
        pl.col('word_length').mean().alias(f'word_length_mean')
    ]

    df = df.group_by(['essay_id'], maintain_order=True).agg(feature_aggregates).sort('essay_id')
    df = df.to_pandas()
    return df

In [6]:
word_feature_df = create_word_features(train_df)
word_feature_df.head()

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:140: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(


,essay_id,word_length_greater_equal_1,word_length_greater_equal_2,word_length_greater_equal_3,word_length_greater_equal_4,word_length_greater_equal_5,word_length_greater_equal_6,word_length_greater_equal_7,word_length_greater_equal_8,word_length_greater_equal_9,...,word_length_greater_equal_12,word_length_greater_equal_13,word_length_greater_equal_14,word_length_greater_equal_15,word_length_max,word_length_mean,word_length_std,word_length_q1,word_length_q2,word_length_q3
0,000d118,491,471,391,289,169,119,76,46,32,...,6,6,5,2,25,4.378819,2.538495,3.0,4.0,5.0
1,000fe60,332,311,248,184,103,63,41,29,16,...,0,0,0,0,11,4.012048,2.060968,2.0,4.0,5.0
2,001ab80,550,539,445,328,200,135,103,72,54,...,14,10,5,2,15,4.574545,2.604621,3.0,4.0,5.0
3,001bdc0,446,431,348,277,217,175,133,85,51,...,9,2,1,1,17,4.997758,2.767791,3.0,4.0,7.0
4,002ba53,373,354,280,213,170,138,112,68,49,...,9,6,2,0,14,4.857909,2.861249,3.0,4.0,7.0


# Create Paragraph Features

In [7]:
def create_paragraph_features(df):
    df = df.with_columns(
        pl.col('full_text')
        .str.split(by='\n\n')
        .alias('paragraph')
    )

    # Expand the paragraph list into several lines of data
    df = df.explode('paragraph')

    df = df.with_columns(
        pl.col('paragraph')
        .map_elements(sanitize_text)
    )

    df = df.with_columns(
        pl.col('paragraph')
        .map_elements(remove_punctuation)
        .map_elements(count_spelling_errors)
        .alias('paragraph_num_errors')
    )

    # Calculate the number of sentences and words in each paragraph
    df = df.with_columns(
        pl.col('paragraph')
        .map_elements(lambda paragraph: len(paragraph))
        .alias('paragraph_num_characters'),
        pl.col('paragraph')
        .map_elements(lambda paragraph: len(paragraph.split(' ')))
        .alias('paragraph_num_words'),
        pl.col('paragraph')
        .map_elements(lambda paragraph: len(paragraph.split('.')))
        .alias('paragraph_num_sentences'),
    )

    # Aggreate features
    feature_aggregates = []

    feature_aggregates += [
        pl.col('paragraph')
        .filter(pl.col('paragraph_num_characters') >= i)
        .count()
        .alias(f'paragraph_num_character_greater_equal_{i}')
        for i in [100, 200, 300, 500, 600, 700]
    ]

    paragraph_feature = ['paragraph_num_characters','paragraph_num_words','paragraph_num_sentences', 'paragraph_num_errors']
    for feature in paragraph_feature:
        feature_aggregates += [
            pl.col(feature).max().alias(f'{feature}_max'),
            pl.col(feature).min().alias(f'{feature}_min'),
            pl.col(feature).mean().alias(f'{feature}_mean'),
        ]

    df = df.group_by(['essay_id'], maintain_order=True).agg(feature_aggregates).sort('essay_id')
    df = df.to_pandas()
    return df

In [8]:
paragraph_feature_df = create_paragraph_features(train_df)
paragraph_feature_df.head()

/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:76: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(
/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:81: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(
/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:81: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. S

,essay_id,paragraph_num_character_greater_equal_0,paragraph_num_character_greater_equal_50,paragraph_num_character_greater_equal_75,paragraph_num_character_greater_equal_100,paragraph_num_character_greater_equal_125,paragraph_num_character_greater_equal_150,paragraph_num_character_greater_equal_175,paragraph_num_character_greater_equal_200,paragraph_num_character_greater_equal_250,...,paragraph_num_sentences_q3,paragraph_num_errors_max,paragraph_num_errors_min,paragraph_num_errors_mean,paragraph_num_errors_sum,paragraph_num_errors_first,paragraph_num_errors_last,paragraph_num_errors_kurtosis,paragraph_num_errors_q1,paragraph_num_errors_q3
0,000d118,1,1,1,1,1,1,1,1,1,...,14.0,23,23,23.000000,23,23,23,NaN,23.0,23.0
1,000fe60,5,5,5,5,5,5,5,4,3,...,5.0,1,0,0.800000,4,1,1,0.250000,1.0,1.0
2,001ab80,4,4,4,4,4,4,4,4,4,...,8.0,2,0,1.250000,5,0,2,-1.371901,1.0,2.0
3,001bdc0,5,5,5,5,5,4,4,4,4,...,8.0,6,0,2.200000,11,2,0,-0.920656,0.0,3.0
4,002ba53,6,4,4,4,4,4,4,4,4,...,5.0,5,0,2.166667,13,0,1,-1.283264,1.0,4.0


# SkLearn

In [9]:
essays = train_df['full_text'].map_elements(sanitize_text)

/tmp/ipykernel_24/1343029068.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  essays = train_df['full_text'].map_elements(sanitize_text)


## Create TF-IDF Features

In [10]:
def create_tfidf_features(df, vectorizer):
    essays = df['full_text'].map_elements(sanitize_text)
    tfidf_sparse_matrix = vectorizer.transform(essays)
        
    tfidf_dense_matrix = tfidf_sparse_matrix.toarray()
    tfidf_df = pd.DataFrame(tfidf_dense_matrix)

    tfidf_df.columns = [f'tfidf_{i}' for i in range(tfidf_df.shape[1])]
    tfidf_df['essay_id'] = df['essay_id']
    return tfidf_df

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_tokenizer(x):
    return x

def tfidf_preprocessor(x):
    return x

tfidf_vectorizer = TfidfVectorizer(
    tokenizer=tfidf_tokenizer,
    preprocessor=tfidf_preprocessor,
    token_pattern=None,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(3, 6),
    min_df=0.05,
    max_df=0.95,
    sublinear_tf=True
)
tfidf_vectorizer.fit(essays)
with open('tfidf_vectorizer.pkl', 'wb') as outfile:
    pickle.dump(tfidf_vectorizer, outfile)

tfidf_df = create_tfidf_features(train_df, tfidf_vectorizer)
tfidf_df.head()

/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:173: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  essays = df['full_text'].map_elements(sanitize_text)


,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,tfidf_6,tfidf_7,tfidf_8,tfidf_9,...,tfidf_19251,tfidf_19252,tfidf_19253,tfidf_19254,tfidf_19255,tfidf_19256,tfidf_19257,tfidf_19258,tfidf_19259,essay_id
0,0.0,0.013906,0.000000,0.027957,0.000000,0.0,0.000000,0.0,0.0,0.014791,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000d118
1,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.049895,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000fe60
2,0.0,0.000000,0.000000,0.000000,0.031504,0.0,0.031526,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,001ab80
3,0.0,0.012879,0.016195,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.015669,0.015782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,001bdc0
4,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,002ba53


## Count Features

In [12]:
def create_bag_of_words_features(df, vectorizer):
    essays = df['full_text'].map_elements(sanitize_text)
    bow_sparse_matrix = vectorizer.transform(essays)

    bow_dense_matrix = bow_sparse_matrix.toarray()
    bow_df = pd.DataFrame(bow_dense_matrix)

    bow_df.columns = [f'bow_{i}' for i in range(bow_df.shape[1])]
    bow_df['essay_id'] = df['essay_id']
    return bow_df

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

def count_tokenizer(x):
    return x

def count_preprocessor(x):
    return x

count_vectorizer = CountVectorizer(
    tokenizer=count_tokenizer,
    preprocessor=count_preprocessor,
    token_pattern=None,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(2,3),
    min_df=0.10,
    max_df=0.85,
)
count_vectorizer.fit(essays)

with open('count_vectorizer.pkl', 'wb') as outfile:
    pickle.dump(count_vectorizer, outfile)

bow_df = create_bag_of_words_features(train_df, count_vectorizer)
bow_df.head()

/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:185: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  essays = df['full_text'].map_elements(sanitize_text)


,bow_0,bow_1,bow_2,bow_3,bow_4,bow_5,bow_6,bow_7,bow_8,bow_9,...,bow_1969,bow_1970,bow_1971,bow_1972,bow_1973,bow_1974,bow_1975,bow_1976,bow_1977,essay_id
0,4,0,1,0,1,0,0,4,1,0,...,3,0,0,0,0,0,0,0,0,000d118
1,4,0,0,0,0,0,0,0,0,0,...,2,0,0,1,1,0,0,0,0,000fe60
2,6,0,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,001ab80
3,2,0,1,0,0,0,3,0,0,0,...,1,0,0,0,0,4,4,0,0,001bdc0
4,0,0,0,0,0,0,3,1,0,0,...,0,0,0,0,0,0,0,0,0,002ba53


In [14]:
import pickle
import numpy as np

train_df = train_df.to_pandas()
for df in [llm_df, paragraph_feature_df, word_feature_df, tfidf_df, bow_df]:
    train_df = train_df.merge(df, on='essay_id', how='left')


with open('train_features.pkl', 'wb') as outfile:
    pickle.dump(train_df, outfile)

In [15]:
feature_names = list(filter(lambda x: x not in ['full_text', 'essay_id','score'], train_df.columns))
X = train_df[feature_names].astype(np.float32).values
y = train_df['score'].astype(np.float32).values - a
labels = train_df['score'].astype(int).values

In [16]:
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from lightgbm import log_evaluation, early_stopping
from sklearn.metrics import f1_score

models = []
results = []
kappa_scores = []

features = feature_names

straified_k_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
splits = straified_k_folds.split(X, labels)
        
for fold_index, (train_indices, validation_indices) in enumerate(splits):
    X_train, X_validation = X[train_indices], X[validation_indices]
    y_train, y_validation = y[train_indices], y[validation_indices], 
    labels_validation = labels[validation_indices]
    
    lgbm_regressor = lgb.LGBMRegressor(
        objective=quadratic_weighted_kappa_objective,
        metrics='None',
        learning_rate=0.05,
        max_depth=5,
        num_leaves=10,
        colsample_bytree=0.3,
        reg_alpha=0.7,
        reg_lambda=0.1,
        n_estimators=700,
        random_state=412,
        extra_trees=True,
        class_weight='balanced',
        device='gpu',
        verbosity=-1
    )

    callbacks = [
        log_evaluation(period=25),
        early_stopping(stopping_rounds=75, first_metric_only=True)
    ]
    
    model = lgbm_regressor.fit(
        X_train,
        y_train,
        eval_names=['train', 'validation'],
        eval_set=[(X_train, y_train), (X_validation, y_validation)],
        eval_metric=quadratic_weighted_kappa_metric,
        callbacks=callbacks
    )

    models.append(model)
    predictions = model.predict(X_validation)
    predictions = (predictions + a).round().clip(1, 6)

    validation_df = train_df.iloc[validation_indices].copy(deep=True)
    validation_df.loc[validation_indices, 'predictions'] = predictions
    results.append(validation_df)
    
    kappa = cohen_kappa_score(labels_validation, predictions, weights='quadratic')
    kappa_scores.append(kappa)
    model.booster_.save_model(f'fold_{fold_index}.txt')

    print(f'Cohen kappa score across fold: {kappa}')

1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.833248	validation's QWK: 0.830172
[50]	train's QWK: 0.874346	validation's QWK: 0.870758
[75]	train's QWK: 0.880185	validation's QWK: 0.878093
[100]	train's QWK: 0.883662	validation's QWK: 0.880765
[125]	train's QWK: 0.885366	validation's QWK: 0.881962
[150]	train's QWK: 0.887876	validation's QWK: 0.881221
[175]	train's QWK: 0.889851	validation's QWK: 0.880932
Early stopping, best iteration is:
[118]	train's QWK: 0.885142	validation's QWK: 0.882652
Evaluated only: QWK
Cohen kappa score across fold: 0.8826524660402125
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.830746	validation's QWK: 0.831361
[50]	train's QWK: 0.865672	validation's QWK: 0.866725
[75]	train's QWK: 0.875914	validation's QWK: 0.87661
[100]	train's QWK: 0.879393	validation's QWK: 0.87778
[125]	tra